In [1]:
import mne
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import pywt
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection  import train_test_split
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from keras import regularizers
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from mne.decoding import CSP
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

2023-05-05 03:44:24.518350: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-05 03:44:24.573768: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-05 03:44:24.574732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 03:44:25.532503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
raw = mne.io.read_raw_gdf('/home/yash/Downloads/eeg/BCICIV_2a_gdf/A08T.gdf')

Extracting EDF parameters from /home/yash/Downloads/eeg/BCICIV_2a_gdf/A08T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


/home/yash/anaconda3/lib/python3.10/contextlib.py:142: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


In [3]:
events, a= mne.events_from_annotations(raw)


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


In [4]:
raw.load_data()

Reading 0 ... 675269  =      0.000 ...  2701.076 secs...


<RawGDF | A08T.gdf, 25 x 675270 (2701.1 s), ~128.8 MB, data loaded>

In [5]:
raw.filter(8., 13., fir_design='firwin')


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter length: 413 samples (1.652 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.8s finished


<RawGDF | A08T.gdf, 25 x 675270 (2701.1 s), ~128.8 MB, data loaded>

In [6]:
raw.info['bads'] += ['EOG-left', 'EOG-central', 'EOG-right']


In [7]:
picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False,
                       exclude='bads')

In [8]:
tmin, tmax = 2., 6.

In [9]:
event_id = dict({'769': 7,'770': 8,'771':9,'772':10})


In [10]:

epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)

Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 1001 original time points ...
1 bad epochs dropped


In [11]:
epochs

Number of events,287
Events,769: 72770: 72771: 71772: 72
Time range,2.000 – 6.000 sec
Baseline,off


In [12]:
data = epochs.get_data()

In [13]:
labels = epochs.events[:,-1] - 7 

In [14]:
data.shape

(287, 22, 1001)

In [49]:
x_train,x_test ,y_train,y_test = train_test_split(data,labels,test_size = 0.25 , random_state =42 , shuffle = True,stratify = labels )

In [50]:
cv = ShuffleSplit(n_splits = 12, test_size = 0.25, random_state = 42)

In [51]:
acc_xgb =[]
acc_random=[]

In [52]:
training_label = y_train
training_data = x_train

In [53]:
pred_xgb = np.zeros((y_test.shape[0],4)).astype('int')
pred_randomforest= np.zeros((y_test.shape[0],4)).astype('int')

In [54]:
for train_idx, val_idx in cv.split(training_label):
    y_train,y_val = training_label[train_idx],training_label[val_idx]
    x_train,x_val = training_data[train_idx],training_data[val_idx]
    n_components = 8
    csp =CSP(n_components)
    mima =MinMaxScaler()
    csp.fit(x_train,y_train)
    x_train_csp= csp.transform(x_train)
    x_val_csp = csp.transform(x_val)
    x_te  = csp.transform(x_test)
    x_train_mi  = mima.fit_transform(x_train_csp)
    x_val_mi  = mima.transform(x_val_csp)
    x_te_mi = mima.transform(x_te)
    xgb =XGBClassifier()
    randomforest =RandomForestClassifier()
    xgb.fit(x_train_mi,y_train)
    randomforest.fit(x_train_mi,y_train)
    y_val_prxgb =xgb.predict(x_val_mi)
    y_val_prrandom =randomforest.predict(x_val_mi)
    y_te_xgb = xgb.predict(x_te_mi)
    y_te_random = randomforest.predict(x_te_mi)
    for h in range(y_te.shape[0]):
        pred_xgb[h,y_te_xgb[h]]+=1
        pred_randomforest[h,y_te_random[h]]+=1
    acc_xgb.append(accuracy_score(y_val_prxgb,y_val))
    acc_random.append(accuracy_score(y_val_prrandom,y_val))
    
    

Computing rank from data with rank=None
    Using tolerance 5.6e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 3.6e-05 (2.2e-16 eps * 22 dim * 7.4e+09  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 3.2e-05 (2.2e-16 eps * 22 dim * 6.6e+09  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 3.6e-05 (2.2e-16 eps * 22 dim * 7.3e+09  max singular value)


Done.
Computing rank from data with rank=None
    Using tolerance 4.6e-05 (2.2e-16 eps * 22 dim * 9.5e+09  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 3.3e-05 (2.2e-16 eps * 22 dim * 6.9e+09  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 3.3e-05 (2.2e-16 eps * 22 dim * 6.9e+09  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5e-05 (2.2e-16 eps * 22 dim * 1e+10  max singular value

In [55]:
acc_xgb

[0.5370370370370371,
 0.5,
 0.5740740740740741,
 0.5185185185185185,
 0.5370370370370371,
 0.5185185185185185,
 0.4444444444444444,
 0.6111111111111112,
 0.46296296296296297,
 0.5185185185185185,
 0.4074074074074074,
 0.48148148148148145]

In [56]:
acc_random

[0.5185185185185185,
 0.46296296296296297,
 0.6111111111111112,
 0.5370370370370371,
 0.5370370370370371,
 0.6111111111111112,
 0.5,
 0.5925925925925926,
 0.5,
 0.5185185185185185,
 0.48148148148148145,
 0.48148148148148145]

In [57]:
y_test

array([3, 3, 0, 2, 3, 3, 0, 0, 3, 3, 0, 2, 0, 2, 0, 1, 3, 3, 1, 1, 2, 3,
       0, 2, 1, 2, 1, 2, 0, 1, 3, 1, 0, 1, 2, 3, 2, 0, 0, 3, 2, 1, 2, 1,
       2, 0, 3, 3, 2, 3, 1, 0, 2, 0, 3, 1, 1, 1, 3, 1, 0, 2, 2, 2, 3, 0,
       1, 1, 0, 1, 0, 2])

In [58]:
pred_xgb

array([[ 0,  0,  0, 12],
       [ 1,  0,  5,  6],
       [ 6,  1,  2,  3],
       [ 5,  1,  6,  0],
       [ 4,  1,  3,  4],
       [ 0,  0,  0, 12],
       [ 3,  1,  8,  0],
       [ 1,  3,  0,  8],
       [ 0,  0,  0, 12],
       [ 0,  0,  1, 11],
       [ 1,  0,  6,  5],
       [ 0,  3,  5,  4],
       [ 9,  2,  1,  0],
       [ 3,  1,  8,  0],
       [ 7,  2,  3,  0],
       [ 6,  6,  0,  0],
       [ 0,  0,  1, 11],
       [ 0,  0,  2, 10],
       [11,  1,  0,  0],
       [ 9,  0,  3,  0],
       [ 0,  1,  1, 10],
       [ 0,  1,  0, 11],
       [ 1, 10,  0,  1],
       [ 0,  0, 12,  0],
       [ 2,  7,  1,  2],
       [ 2,  1,  9,  0],
       [ 0,  2,  0, 10],
       [ 5,  0,  7,  0],
       [ 9,  1,  2,  0],
       [ 0,  0, 12,  0],
       [ 0,  0,  0, 12],
       [ 2,  0,  6,  4],
       [ 5,  2,  5,  0],
       [ 0,  6,  6,  0],
       [ 0,  3,  6,  3],
       [ 0,  0,  0, 12],
       [ 8,  1,  3,  0],
       [ 2,  9,  0,  1],
       [ 2,  1,  7,  2],
       [ 0,  0,  0, 12],


In [59]:
accuracy_score(pred_xgb.argmax(axis=1),y_test)

0.6111111111111112

In [61]:
accuracy_score(pred_randomforest.argmax(axis=1),y_test)

0.5694444444444444